In [3]:
import pandas as pd
import re
from fuzzywuzzy import process, fuzz
import numpy as np
import sqlite3
from datetime import datetime

extracted_data_fname = 'product_raw_data_2023-11-27.csv'
raw_data_df = pd.read_csv(extracted_data_fname)

# Remove all trailers and lowercase all string
raw_data_df = raw_data_df.apply(lambda x: x.str.lower().str.strip() if x.dtype == 'object' else x)

# Clean the product_name column
raw_data_df['product_name'] = raw_data_df['product_name'].astype(object) # Specify the datatype as object
raw_data_df = raw_data_df[raw_data_df['product_name'].notnull()] # product_name field should have no null values

# Clean the product_name column
raw_data_df['product_name'] = raw_data_df['product_name'].astype(object) # Specify the datatype as object
raw_data_df = raw_data_df[raw_data_df['product_name'].notnull()] # product_name field should have no null values

regex_volume = re.compile(r'(\d+cl|\d+ml|\d+l|\d+.liter|\d+.ml|\d+.cl)') # Look for volume on product_name

raw_data_df['volume'] = raw_data_df['product_name'].str.extractall(regex_volume).astype(str).groupby(level=0).agg(','.join) # Create another column for volume and concatenate them if volume variety is more than 1

raw_data_df['product_name'] = raw_data_df['product_name'].apply(lambda x: regex_volume.sub('', x)) # remove volumes on the product_name column

# Add column freebies since some products has freebies
regex_freebie = re.compile(r'(with.+|w/.+|free.*|.*not for sale.*)')
raw_data_df['freebie'] = raw_data_df['product_name'].str.extract(regex_freebie).astype(str).groupby(level=0).agg(','.join) # Create another column for volume and concatenate them if freebies variety is more than 1
raw_data_df['product_name'] = raw_data_df['product_name'].apply(lambda x: regex_freebie.sub('', x)) # remove freebies on the product_name column

regex_gift = re.compile(r'(festive gift.+|gift .+|gifting kit)') 
raw_data_df['gift_set'] = raw_data_df['product_name'].str.extract(regex_gift).astype(str).groupby(level=0).agg(','.join) # Create another column for gift_set
raw_data_df['product_name'] = raw_data_df['product_name'].apply(lambda x: regex_gift.sub('', x)) # remove gift sets on product_name_column

regex_limited = re.compile(r'(select limited edition gift packaging|edition moira gift box|limited.+)') 
raw_data_df['limited_edition'] = raw_data_df['product_name'].str.extract(regex_limited).astype(str).groupby(level=0).agg(','.join) # Create another column for limited edition
raw_data_df['product_name'] = raw_data_df['product_name'].apply(lambda x: regex_limited.sub('', x)) # remove limited edition on product_name_column

raw_data_df['product_name'] = raw_data_df['product_name'].str.replace('[^a-z0-9\s%.&]', '', regex = True) # Remove special characters except %, . and &
raw_data_df = raw_data_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

raw_data_df['product_name'] = raw_data_df['product_name'].replace('', 'not for sale') # after removing freebies on the column some has become empty strings since it is not for sale.

raw_data_df.loc[raw_data_df['product_name'] == 'chivas regal 12 year old blended scotch whisky', 'product_name'] = 'chivas regal 12 years old blended scotch whisky'
raw_data_df.loc[raw_data_df['product_name'] == 'maria clara .', 'product_name'] = 'maria clara'


# Extract brand name on product_name column Using fuzzy partial_ratio
# Remove special charactes on brand
raw_data_df['brand'] = raw_data_df['brand'] = raw_data_df['brand'].str.replace('[^a-z0-9\s&]', '', regex = True) # Remove special characters except % and .

existing_brands = raw_data_df['brand'].unique() # get the unique value for the brand column
additional_brands = pd.read_csv('alcohol_brand_names.csv') # import available data set online for liquor brands all over the world. add brand if needed
additional_brands['Brand_Name'] = additional_brands['Brand_Name'].str.lower() # Make sure the brands are in lower case

# Check if additional_brands is empty
if not additional_brands.empty:
    combined_unique_brands = pd.unique(pd.concat([pd.Series(existing_brands), additional_brands['Brand_Name']]))
else:
    combined_unique_brands = existing_brands

products = raw_data_df['product_name']


best_matches = []
scores = []

for product in products:
    # Convert each brand to string to handle non-string elements
    brand_str = [str(combined_unique_brand) for combined_unique_brand in combined_unique_brands]

    # Find the best match for the current product in the list of brands using partial_ratio
    result = process.extractOne(product, brand_str, scorer=fuzz.partial_ratio)

    # The result is a tuple with the best match and its score
    best_match, score = result


    # Append the results to the lists
    best_matches.append(best_match)
    scores.append(score)

# Add the lists as new columns to the DataFrame
raw_data_df['best_brand_match'] = best_matches
raw_data_df['similarity_score'] = scores

# Update the 'brand' column based on the matching result if the 'brand' is null and the score is greater than or equal to 89
condition = (raw_data_df['brand'].isna()) & (raw_data_df['similarity_score'] >= 89) # adjust the similarity score as needed
raw_data_df.loc[condition, 'brand'] = raw_data_df.loc[condition, 'best_brand_match']

# Drop the colulumn best_brand_match and similarity_score
raw_data_df.drop(['best_brand_match', 'similarity_score'], axis=1, inplace=True)

# Check if 'brand' is null and fill with 'unknown brand'
raw_data_df['brand'].fillna('none liquor brand', inplace=True)

# Change chivas as chivas regal in brand_column
raw_data_df['brand'] = raw_data_df['brand'].replace('chivas', 'chivas regal')

# Change the datatype of extraction_date column fro object to datetime
raw_data_df['extraction_date'] = pd.to_datetime(raw_data_df['extraction_date'])


# split volume into two columns volume_quantity and volume_unit
raw_data_df['volume'] = raw_data_df['volume'].str.replace(" ", "")
regex_volume_unit =  re.compile(r'(\d+)\s*([a-zA-Z]+)')
matches = raw_data_df['volume'].str.extractall(regex_volume_unit)

# Assign the results to separate columns
raw_data_df['volume_quantity'] = matches[0].astype(str).groupby(level=0).agg(','.join)
raw_data_df['volume_unit'] = matches[1].astype(str).groupby(level=0).agg(','.join)

# Drop the column volume
raw_data_df.drop(['volume'], axis=1, inplace=True)

# remove special characters and headers, trailers on frebie column
raw_data_df['freebie'] = raw_data_df['freebie'].str.replace('[^a-z0-9\s&/]', '', regex = True) # Remove special characters except & and /
raw_data_df['freebie'] = raw_data_df['freebie'].str.strip()

# remove special characters and headers, trailers on gift_set column
raw_data_df['gift_set'] = raw_data_df['gift_set'].str.replace('[^a-z0-9\s]', '', regex = True) # Remove all special characters 
raw_data_df['gift_set'] = raw_data_df['gift_set'].str.strip()

# remove special characters and headers, trailers onv limited_edition column
raw_data_df['limited_edition'] = raw_data_df['limited_edition'].str.replace('[^a-z0-9\s%]', '', regex = True) # Remove all special characters except % 
raw_data_df['limited_edition'] = raw_data_df['limited_edition'].str.strip()

# replace all null values as nan
null_values = [None, 'NA', '', "['']", 'nan']
raw_data_df = raw_data_df.replace(null_values, np.nan)

# remove special characters except ','
raw_data_df['variation'] = raw_data_df['variation'].str.replace('[^a-z0-9\s,]', '', regex = True) 

# Change data type of current_discount_percentage and volume unit as int 
raw_data_df['current_discount_percentage'] = raw_data_df['current_discount_percentage'].str.rstrip('%')
raw_data_df['current_discount_percentage'] = raw_data_df['current_discount_percentage'].astype(float)


# Round of the column shop_rating
raw_data_df['shop_rating']= raw_data_df['shop_rating'].round(decimals=4)


# export raw_data_df as sqlite for database flat/wide table design
db_clean_filename = f"product_clean_data_2023-11-27.db"
conn = sqlite3.connect(db_clean_filename) # Connect to SQLite db

# specify the column data types 
column_data_types = {
    "unique_item_id": "INTEGER PRIMARY KEY",
    "product_name": "TEXT",
    "current_stock": "INTEGER",
    "shop_name": "TEXT",
    "shop_id": "INTEGER",
    "brand": "TEXT",
    "sold_per_month": "INTEGER",
    "historical_sold": "INTEGER",
    "liked_count": "NUMERIC",
    "variation_type": "TEXT",
    "variation": "TEXT",
    "current_price": "NUMERIC",
    "min_price": "NUMERIC",
    "max_price": "NUMERIC",
    "lowest_price_guarantee": "INTEGER",
    "current_discount_percentage": "NUMERIC",
    "rating_star": "NUMERIC",
    "shopee_verified": "INTEGER",
    "official_shop": "INTEGER",
    "cc_installment": "INTEGER",
    "none_cc_installment": "INTEGER",
    "preferred_seller": "INTEGER",
    "shop_location": "TEXT",
    "shop_rating": "NUMERIC",
    "cod": "TEXT",
    "extraction_date": "TEXT",
    "freebie": "TEXT",
    "gift_set": "TEXT",
    "limited_edition": "TEXT",
    "volume_quantity": "NUMERIC",
    "volume_unit": "TEXT",
}

raw_data_df.to_sql('products', conn, index=False, if_exists='replace', dtype=column_data_types) # save to sql database
conn.close()


# save to csv format
clean_excel_fname = f"product_clean_data_{datetime.now().strftime('%Y-%m-%d')}.xlsx"
raw_data_df.to_excel(clean_excel_fname, index=False)